<a href="https://colab.research.google.com/github/yk-Jeong/dsnote/blob/main/9%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. lib and data load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
df = pd.read_csv(data)

In [ ]:
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


mileage, engine, max_power -> 단위 제거해 수치형으로

torque 분할 

fuel, seller_type, transmission -> 원핫인코딩

owner -> 단계별 변수로 

In [ ]:
# 이상치 확인 가능 

round(df.describe(), 2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


### 1. EDA 

In [ ]:
# engine

df['engine'].str.split(expand=True)

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [ ]:
# 단위는 모두 CC로 일치하는 것을 알 수 있음 

df['engine'].str.split(expand=True)[1].unique()

array(['CC', nan], dtype=object)

In [ ]:
# 문자 표기 삭제 후 숫자형으로 변경, 컬럼명에 단위명을 표기 

df['engine'] = df['engine'].str.split(expand=True)[0].astype('float32')
df.rename(columns = {'engine':'engine_cc'}, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine_cc      7907 non-null   float32
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float32(1), float64(1), int64(3), object(8)
memory usage: 793.9+ KB


max_power_bhp 항목에서 에러 발생해 해결 

In [ ]:
df[df['max_power'] == ' bhp']

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine_cc,max_power,torque,seats
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0


In [ ]:
def isFloat(value):
  try:
    num = float(value)
    return num
  except ValueError:
    return np.NaN # numpy null 값으로 적용

In [ ]:
# applpy 함수를 활용해 df에 적용하고 결측치 삭제 

df['max_power'] = df['max_power'].apply(isFloat)
df.drop('max_power', axis=1, inplace=True)

mileage의 경우, 별도의 단위가 존재 -> fuel 변수를 활용해 단위 통일

In [ ]:
df['mileage'].str.split(expand=True)[1].unique()

array(['kmpl', 'km/kg', nan], dtype=object)

In [ ]:
df['fuel'].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [ ]:
df['mileage'] = df['mileage'].str.split(expand=True)[0].astype('float32')

In [ ]:
def mile(df):
  if df['fuel'] == 'Diesel':
    return df['mileage'] / 73.56
  elif df['fuel'] == 'Petrol':
    return df['mileage'] / 80.43
  elif df['fuel'] == 'LPG':
    return df['mileage'] / 40.85
  else:
    return df['mileage'] / 44.23

In [ ]:
# 수치형으로 수정 후, 리터당 가격으로 변경

df['mileage'] = df['mileage'].str.split(expand=True)[0].astype('float32')
df['mileage'] = df.apply(mile, axis=1)

torque 

In [ ]:
df['torque'].head()

0              190Nm@ 2000rpm
1         250Nm@ 1500-2500rpm
2       12.7@ 2,700(kgm@ rpm)
3    22.4 kgm at 1750-2750rpm
4       11.5@ 4,500(kgm@ rpm)
Name: torque, dtype: object

In [ ]:
df['torque'] = df['torque'].str.lower()

In [ ]:
# 단위 변환을 위한 함수 선언 후 적용

def torque_unit(x):
  if 'nm' in str(x):
    return 'Nm'
  elif 'kgm'in str(x):
    return 'kgm'

df['torque_unit'] = df['torque'].apply(torque_unit)

In [ ]:
df['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [ ]:
# Nm, kgm 이외의 단위가 존재하는지 확인 

df[df['torque_unit'].isna()]['torque'].unique()

array([nan, '250@ 1250-5000rpm', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)